In [7]:
from datetime import datetime
import pandas as pd
import numpy as np
import cv2
import os

def get_timestamp_from_filename(filename):
    timestamp_str = filename.split(".")[0][4:23]
    try:
        return datetime.strptime(timestamp_str, "%Y-%m-%d-%H-%M-%S")
    except ValueError:
        return None

def extract_frames(video_path: str):
    cap = cv2.VideoCapture(video_path)
    frames = []
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return np.array(frames)

def is_in_zone(x:int, y:int, zone:np.ndarray) -> bool:
    return cv2.pointPolygonTest(zone, (int(x), int(y)), measureDist=False) >= 0

def is_in_zone_vec(centers:np.ndarray, zone:np.ndarray) -> np.ndarray:
    return np.array([
        is_in_zone(x, y, zone)
        for x, y in centers
    ]).astype(bool)

parking_region_pix = np.array([
    [545, 129],
    [674, 156],
    [774, 291],
    [610, 263],
])
driving_region_pix = np.array([
    [357, 60],
    [522, 85],
    [666, 385],
    [401, 385],
])

def take_drive_pics(record, vid_path, drive_pics_folder_path, frame_skip=1):
    os.makedirs(drive_pics_folder_path, exist_ok=True)
    
    record = record.copy()
    record['in_driving_region'] = is_in_zone_vec(record[['cx', 'cy']].values.astype(int), driving_region_pix)
    frames = extract_frames(vid_path)
    
    for i, row in record.iterrows():
        if row['in_driving_region'] and row['iteration'] % frame_skip == 0:
            frame_idx = row['iteration']
            if frame_idx < len(frames):  # Add this check
                frame = frames[frame_idx]
            
            if not np.isnan(row["tl_y"]) and not np.isnan(row["tl_x"]) and not np.isnan(row["h"]) and not np.isnan(row["w"]):
                cropped = frame[int(row["tl_y"]):(int(row["tl_y"]) + int(row["h"])),
                                int(row["tl_x"]):(int(row["tl_x"]) + int(row["w"]))]
                cv2.imwrite(os.path.join(drive_pics_folder_path, f"frame{i}.jpg"), cropped)
    if len(os.listdir(drive_pics_folder_path)) == 0:
        os.rmdir(drive_pics_folder_path)

In [8]:
data = pd.read_csv('../../zoning_data/video_information.csv')
license_plates = set(data["fake_license_plate"])

all_cars = pd.DataFrame(columns=["time", "license_plate"])

for plate in license_plates:
    car_data = data[data["fake_license_plate"] == plate]
    if len(car_data) == 2:
        if not os.path.exists(f"../test_data/crabtree_lot_4/{plate}"):
            os.makedirs(f"../test_data/crabtree_lot_4/{plate}")
        
        enter_vid = car_data[car_data["action"] == "enter"].iloc[0]["enter_video"]
        leave_vid = car_data[car_data["action"] == "leave"].iloc[0]["leave_video"]
        time = get_timestamp_from_filename(enter_vid)

        enter_vid_path = f"../../zoning_data/{enter_vid[:14]}/MP4s/{enter_vid}"
        enter_record_path = enter_vid_path.replace(".mp4", ".csv")
        enter_record_path = enter_record_path.replace("MP4s", "CSVs")
        enter_record = pd.read_csv(enter_record_path)
        rel_enter_record = enter_record[enter_record["track_id"] == car_data[car_data["action"] == "enter"].iloc[0]["track_id"]]
        
        leave_vid_path = f"../../zoning_data/{leave_vid[:14]}/MP4s/{leave_vid}"
        leave_record_path = leave_vid_path.replace(".mp4", ".csv")
        leave_record_path = leave_record_path.replace("MP4s", "CSVs")
        leave_record = pd.read_csv(leave_record_path)
        rel_leave_record = leave_record[leave_record["track_id"] == car_data[car_data["action"] == "leave"].iloc[0]["track_id"]]
        
        enter_pics_folder_path = f"../test_data/crabtree_lot_4/{plate}/enter/"
        leave_pics_folder_path = f"../test_data/crabtree_lot_4/{plate}/leave/"
        
        take_drive_pics(rel_enter_record, enter_vid_path, enter_pics_folder_path, frame_skip=4)
        take_drive_pics(rel_enter_record, leave_vid_path, leave_pics_folder_path, frame_skip=4)
        
        if len(os.listdir(f"../test_data/crabtree_lot_4/{plate}")) == 0:
            os.rmdir(f"../test_data/crabtree_lot_4/{plate}")
        else: 
            new_car = pd.DataFrame({"time": [time], "license_plate": [plate]}, index=[0])
            all_cars = pd.concat([all_cars, new_car], ignore_index=True)
    
all_cars.to_csv("../test_data/crabtree_lot_4/all_cars.csv", index=False)

/var/folders/b7/rm2hp2x501zcmrphlxfn29gh0000gn/T/ipykernel_89560/68731093.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_cars = pd.concat([all_cars, new_car], ignore_index=True)
